# Cal-Adapt Analytics Engine: Threshold Tools Application Examples

A notebook on specific threshold-related applications with the *climakitae* package and *threshold_tools*. The notebook walks through both basic and advanced topics and covers capabilities such as evaluating goodness of fit, calculating return values and return periods, subsetting and filtering data, observing trends through time, and more.

The notebook focuses on two major *threshold_tools* applications: **updating design standards** and **asset-by-asset vulnerability assessments**, which are highlighted as application examples throughout the notebook.

### Step 0: Import

#### Import neccessary packages before running analysis

In [ ]:
import xarray as xr
import panel as pn
pn.extension()

In [ ]:
!pip install git+https://github.com/OpenHydrology/lmoments3.git

In [ ]:
import climakitae as ck
from climakitae import threshold_tools

## Threshold Basics: Exploring Applications At County-Level

### Step 1: Select

#### Load a new application and call *select* to display interface from which to choose location, variables, scenarios, and designate warming levels of interest

In [ ]:
app = ck.Application()

#### Call *select* to display an interface from which to select the data to examine

For this section, please select:
- "daily" timescale,
- "2m Air Temperature",
- "9 km" resolution,
- "SSP3-7.0 -- Business as Usual" scenario with "Append historical",
- and "Ca counties" area subset with "Scramento County" cached area.

In [ ]:
app.select()

### Step 2: Retrieve

#### Call *app.retrieve()* to load the subset/combo of data specified

In [ ]:
sacramento_ds = app.retrieve()
sacramento_ds

#### Convert default units to desired units

Note: Unit conversions are set to be built as a capability into *climakitae* package, so eventually manual coded conversions will not be necessary

In [ ]:
sacramento_ds.data = sacramento_ds.data - 273.15
sacramento_ds.attrs['units'] = '˚C'

#### Subset data by scenario and simulation to prepare it for *threshold_tools* functions

Note: Currently threshold_tools requires a dataarray where this is only 1 simulation and 1 scenario selected

In [ ]:
sacramento_da = sacramento_ds.sel(simulation='cnrm-esm2-1')

### Step 3: Transform

#### Pull Annual Maximum Series (AMS) for all grid cells

In [ ]:
sacramento_ams = threshold_tools.get_ams(sacramento_da, extremes_type='max')
sacramento_ams = sacramento_ams.compute()

#### Subset data by time to prepare it for specific application

In [ ]:
sacramento_1980_ams = sacramento_ams.sel(time=slice('1980-01-01', '2010-01-01'))

#### Utilize KS test to calculate goodness of fit of selected distribution

<span style="color:#E47704">

**Application Example:** A electric utility in Sacramento wants to ensure that the return value and return period results they calculate are "statistically sound" and appropriate to use for their asset vulnerability assessments and design standards. 
    
For instance, is the GEV probability distribution a good fit for the data and the right probabibility distribution to calculate return values and return periods?

In [ ]:
sacramento_1980_ks = threshold_tools.get_ks_stat(sacramento_1980_ams, distr='gev', multiple_points=True)
sacramento_1980_ks

#### Calculate return value for a selected return period

<span style="color:#E47704">
    
**Application Example:** A electric utility planning on building new electrical equipment in Sacramento wants to calculate the value of a 1-in-20-year extreme temperature event that occurred historically (during the 1980-2010 time period) as a benchmark input for updating the design standards of new equipment.

In [ ]:
sacramento_1980_rv = threshold_tools.get_return_value(sacramento_1980_ams, return_period=20, 
                                                      distr='gev', bootstrap_runs=100, 
                                                      conf_int_lower_bound=2.5, 
                                                      conf_int_upper_bound=97.5, 
                                                      multiple_points=True)
sacramento_1980_rv

#### Calculate return period for a selected return value

<span style="color:#E47704">
    
**Application Example:** A electric utility with existing electrical infrastructure in Sacramento wants to calculate the return period of a 35 degrees C temperature event that occurred historically (during the 1980-2010 time period) as a benchmark input for their reoccuring asset vulnerability assessment.

In [ ]:
sacramento_1980_rp = threshold_tools.get_return_period(sacramento_1980_ams, return_value=35, 
                                                       distr='gev', bootstrap_runs=100, 
                                                       conf_int_lower_bound=2.5, 
                                                       conf_int_upper_bound=97.5, 
                                                       multiple_points=True)
sacramento_1980_rp

### Step 4: Visualize

#### Visualize goodness of fit of distribution

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_1980_ks, data_variable='p_value')

#### Visualize return value

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_1980_rv, data_variable='return_value')

#### Visualize return period

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_1980_rp, data_variable='return_period', bar_max=100)

## Threshold Advanced: Exploring Variations At County-Level

### Step 3: Transform

#### Subset data by time to prepare it for specific application

In [ ]:
sacramento_2020_ams = sacramento_ams.sel(time=slice('2020-01-01', '2050-01-01'))
sacramento_2050_ams = sacramento_ams.sel(time=slice('2050-01-01', '2080-01-01'))

#### Calculate return value for a selected return period

<span style="color:#E47704">
    
**Application Example:** A electric utility planning on building new electrical equipment in Sacramento wants to calculate the value of a 1-in-20-year extreme temperature event that will occur in the future (during the 2020-50 and 2050-80 time periods) to ensure:
- that the planned equipment has the appropriate design standards to withstand extreme temperature events in the future,
- and to update the design standards for any future equipment built.

In [ ]:
sacramento_2020_rv = threshold_tools.get_return_value(sacramento_2020_ams, return_period=20,
                                                distr='gev', bootstrap_runs=100, 
                                                conf_int_lower_bound=2.5, 
                                                conf_int_upper_bound=97.5, 
                                                multiple_points=True)
sacramento_2020_rv

In [ ]:
sacramento_2050_rv = threshold_tools.get_return_value(sacramento_2050_ams, return_period=20,
                                                distr='gev', bootstrap_runs=100, 
                                                conf_int_lower_bound=2.5, 
                                                conf_int_upper_bound=97.5, 
                                                multiple_points=True)
sacramento_2050_rv

#### Calculate return period for a selected return value

<span style="color:#E47704">
    
**Application Example:** A electric utility with existing electrical infrastructure in Sacramento wants to calculate the return period of a 44 degrees C temperature event that that will occur in the future (during the 2020-50 and 2050-80 time periods) to understand:
- if the existing infrastructure will be impacted by more frequently occurring extreme temperature events in the future,
- and to complete a more robust asset-by-asset vulnerability assessment.
    
For the utility, 33 degrees C (hypothetically) repersents a county-wide historic average for a 1-in-20-year extreme temperature event (during the 1980-2010 time period).

In [ ]:
sacramento_1980_rp = threshold_tools.get_return_period(sacramento_1980_ams, return_value=33,
                                                       distr='gev', bootstrap_runs=100, 
                                                       conf_int_lower_bound=2.5, 
                                                       conf_int_upper_bound=97.5, 
                                                       multiple_points=True)
sacramento_1980_rp

In [ ]:
sacramento_2020_rp = threshold_tools.get_return_period(sacramento_2020_ams, return_value=33,
                                                       distr='gev', bootstrap_runs=100, 
                                                       conf_int_lower_bound=2.5, 
                                                       conf_int_upper_bound=97.5, 
                                                       multiple_points=True)
sacramento_2020_rp

In [ ]:
sacramento_2050_rp = threshold_tools.get_return_period(sacramento_2050_ams, return_value=33,
                                                       distr='gev', bootstrap_runs=100, 
                                                       conf_int_lower_bound=2.5, 
                                                       conf_int_upper_bound=97.5, 
                                                       multiple_points=True)
sacramento_2050_rp

### Step 4: Visualize

#### Visualize return value

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_1980_rv, data_variable='return_value',
                                    bar_min=25, bar_max=50)

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_2020_rv, data_variable='return_value',
                                    bar_min=25, bar_max=50)

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_2050_rv, data_variable='return_value',
                                    bar_min=25, bar_max=50)

#### Visualize return period

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_1980_rp, data_variable='return_period', 
                                    bar_min=1, bar_max=100)

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_2020_rp, data_variable='return_period',
                                    bar_min=1, bar_max=100)

In [ ]:
threshold_tools.get_geospatial_plot(sacramento_2050_rp, data_variable='return_period',
                                    bar_min=1, bar_max=100)

## Threshold Advanced: Exploring Variations Statewide

### Step 1: Select

#### Call *select* to display an interface from which to select the data to examine

For this section, please select:
- "daily" timescale,
- "2m Air Temperature",
- "9 km" resolution,
- "SSP3-7.0 -- Business as Usual" scenario with "Append historical",
- and "states" area subset with "CA" cached area.

In [ ]:
app.select()

### Step 2: Retrieve

#### Call *app.retrieve()* to load the subset/combo of data specified

In [ ]:
ca_ds = app.retrieve()
ca_ds

#### Convert default units to desired units

Note: Unit conversions are set to be built as a capability into *climakitae* package, so eventually manual coded conversions will not be necessary

In [ ]:
ca_ds.data = ca_ds.data - 273.15
ca_ds.attrs['units'] = '˚C'

#### Subset data by scenario and simulation to prepare it for *threshold_tools* functions

In [ ]:
ca_da = ca_ds.sel(simulation='cnrm-esm2-1')

### Step 3: Transform

#### Pull Annual Maximum Series (AMS) for all grid cells

In [ ]:
ca_ams = threshold_tools.get_ams(ca_da, extremes_type='max')
ca_ams = ca_ams.compute()

#### Subset data by time to prepare it for specific application

In [ ]:
ca_1980_ams = ca_ams.sel(time=slice('1980-01-01', '2010-01-01'))
ca_2020_ams = ca_ams.sel(time=slice('2020-01-01', '2050-01-01'))
ca_2050_ams = ca_ams.sel(time=slice('2050-01-01', '2080-01-01'))

#### Calculate return value for a selected return period

<span style="color:#E47704">
    
**Application Example:** A electric utility planning on building new electrical eqiupment across California wants to calculate the value of a 1-in-20-year extreme temperature event that occured historically (during the 1980-2010 time period) and will occur in the future (during the 2020-50 and 2050-80 time periods) to ensure:
- they can build planned equipment in areas where the equipment does have the appropriate design standards to withstand extreme temperature events in the future,
- and to update the design standards for any future equipment built so that it can account for the range of extreme temperature events across the state.

In [ ]:
ca_1980_rv = threshold_tools.get_return_value(ca_1980_ams, return_period=20,
                                              distr='gev', bootstrap_runs=100, 
                                              conf_int_lower_bound=2.5, 
                                              conf_int_upper_bound=97.5, 
                                              multiple_points=True)
ca_1980_rv

In [ ]:
ca_2020_rv = threshold_tools.get_return_value(ca_2020_ams, return_period=20,
                                              distr='gev', bootstrap_runs=100, 
                                              conf_int_lower_bound=2.5, 
                                              conf_int_upper_bound=97.5, 
                                              multiple_points=True)
ca_2020_rv

In [ ]:
ca_2050_rv = threshold_tools.get_return_value(ca_2050_ams, return_period=20,
                                              distr='gev', bootstrap_runs=100, 
                                              conf_int_lower_bound=2.5, 
                                              conf_int_upper_bound=97.5, 
                                              multiple_points=True)
ca_2050_rv

#### Apply a filter to showcase which areas of California exceed a certain return value

<span style="color:#E47704">
    
**Application Example:** A electric utility is worried about the climate impacts to their solar farms statewide given their current design standards and wants to evaluate those standards for any existing solar farms and update design standards for any future ones. 
    
Their solar farms are made up of crystalline silicon solar cells, which undergo power decreases when ambient air temperature exceeds 25 degrees C (output decreases by 0.66% per 1 degrees C above that threshold).
    
In order to evaluate and update design standards, they want to look at the value of a 1-in-20-year extreme temperature event that will exceeds 40.15 deggrees C (corresponding to a 10% decrease in power output) both historically (during the 1980-2010 time period) and into the future (during the 2020-50 and 2050-80 time periods).

In [ ]:
ca_1980_rv_filtered = ca_1980_rv.where(ca_1980_rv.return_value >= 40.15)

In [ ]:
ca_2020_rv_filtered = ca_2020_rv.where(ca_2020_rv.return_value >= 40.15)

In [ ]:
ca_2050_rv_filtered = ca_2050_rv.where(ca_2050_rv.return_value >= 40.15)

#### Calculate return period for a selected return value

<span style="color:#E47704">
    
**Application Example:** An electric utility with existing electrical infrastructure across California wants to calculate the return period of a 44 degrees C temperature event that occured historically (during the 1980-2010 time period) and will occur in the future (during the 2020-50 and 2050-80 time periods) to understand:
- what areas with existing infrastructure will be impacted by more frequently occuring extreme temperature events in the future,
- and to complete a more robust asset-by-asset vulnerability assessment statewide.
    
For the utility, 33 degrees C (hypothetically) repersents a state-wide historic average for a 1-in-20-year extreme temperature event (during the 1980-2010 time period).

In [ ]:
ca_1980_rp = threshold_tools.get_return_period(ca_1980_ams, return_value=33,
                                               distr='gev', bootstrap_runs=100, 
                                               conf_int_lower_bound=2.5, 
                                               conf_int_upper_bound=97.5, 
                                               multiple_points=True)
ca_1980_rp

In [ ]:
ca_2020_rp = threshold_tools.get_return_period(ca_2020_ams, return_value=33,
                                               distr='gev', bootstrap_runs=100, 
                                               conf_int_lower_bound=2.5, 
                                               conf_int_upper_bound=97.5, 
                                               multiple_points=True)
ca_2020_rp

In [ ]:
ca_2050_rp = threshold_tools.get_return_period(ca_2050_ams, return_value=33,
                                               distr='gev', bootstrap_runs=100, 
                                               conf_int_lower_bound=2.5, 
                                               conf_int_upper_bound=97.5, 
                                               multiple_points=True)
ca_2050_rp

### Step 4: Visualize

#### Visualize return value

In [ ]:
threshold_tools.get_geospatial_plot(ca_1980_rv_filtered, data_variable='return_value',
                                    bar_min=44, bar_max=50)

In [ ]:
threshold_tools.get_geospatial_plot(ca_2020_rv_filtered, data_variable='return_value',
                                    bar_min=44, bar_max=50)

In [ ]:
threshold_tools.get_geospatial_plot(ca_2050_rv_filtered, data_variable='return_value',
                                    bar_min=44, bar_max=50)

#### Visualize return period

In [ ]:
threshold_tools.get_geospatial_plot(ca_1980_rp, data_variable='return_period',
                                    bar_min=1, bar_max=100)

In [ ]:
threshold_tools.get_geospatial_plot(ca_2020_rp, data_variable='return_period',
                                    bar_min=1, bar_max=100)

In [ ]:
threshold_tools.get_geospatial_plot(ca_2050_rp, data_variable='return_period',
                                    bar_min=1, bar_max=100)

### Step 5: Export

Use the below code to export a dataset as a NetCDF, GeoTIFF, or CSV file.

In [ ]:
app.export_as()

In [ ]:
app.export_dataset(return_period,'my_filename')